In [1]:
import os, json, cv2, torch
import numpy as np
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# YOLOv5
from ultralytics import YOLO

# EfficientDet
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchPredict

# COCO API
try:
    from pycocotools.coco import COCO
    from pycocotools.cocoeval import COCOeval
    COCO_AVAILABLE = True
except ImportError:
    print("pycocotools 없음")
    COCO_AVAILABLE = False

import matplotlib.font_manager as fm

In [2]:
# 경로 설정
BASE_DIR = Path.cwd().parent
IMG_DIR = BASE_DIR / "data/raw/images"
JSON_DIR = BASE_DIR / "data/raw/json_labels"
DATASET_YOLO = BASE_DIR / "model/yolov5"
DATASET_EFFDET = BASE_DIR / "model/efficientdet-pytorch"
RESULT_DIR = BASE_DIR / "data/processed/results_comparison"
YOLO_WEIGHTS_FILE = RESULT_DIR / "yolov5su.pt"

for d in [DATASET_YOLO, DATASET_EFFDET, RESULT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

In [3]:
# 한국어 폰트
def setup_korean_font():
    try:
        # Windows
        if os.name == 'nt':
            font_path = 'C:/Windows/Fonts/malgun.ttf'
            if os.path.exists(font_path):
                font_name = fm.FontProperties(fname=font_path).get_name()
                plt.rc('font', family=font_name)
            else:
                plt.rc('font', family='DejaVu Sans')
        # Mac
        elif os.name == 'posix':
            plt.rc('font', family='AppleGothic')
        
        plt.rcParams['axes.unicode_minus'] = False
        print("한글 폰트 설정 완료")
    except Exception as e:
        print(f"폰트 설정 실패: {e}")
        plt.rc('font', family='DejaVu Sans')

setup_korean_font()

한글 폰트 설정 완료


In [4]:
# 데이터 전처리
def preprocess_data():
    jsons = list(JSON_DIR.glob("*.json"))
    if not jsons:
        return {}, []
    
    train, temp = train_test_split(jsons, train_size=0.8, random_state=42)
    val, test = train_test_split(temp, train_size=0.5, random_state=42)
    splits = {'train': [], 'val': [], 'test': []}
    classes, class_to_idx = [], {}
    
    for split, files in zip(['train', 'val', 'test'], [train, val, test]):
        for j in tqdm(files, desc=f"Loading {split}"):
            with open(j, 'r', encoding='utf-8') as f:
                d = json.load(f)

            img_path = None
            for ext in ['.jpg', '.png', '.jpeg', '.JPG', '.PNG', '.JPEG']:
                p = IMG_DIR / f"{j.stem}{ext}"
                if p.exists():
                    img_path = str(p)
                    break
            if not img_path:
                continue
            
            name = f"{d['cate1']}_{d['cate3']}"
            if name not in classes:
                class_to_idx[name] = len(classes)
                classes.append(name)
            
            bbox = d['bndbox']
            splits[split].append({
                'image': img_path,
                'bbox': [bbox['xmin'], bbox['ymin'], bbox['xmax'], bbox['ymax']],
                'label': class_to_idx[name],
                'json_stem': j.stem
            })
    
    print(f"Dataset: train={len(splits['train'])}, val={len(splits['val'])}, test={len(splits['test'])}")
    return splits, classes

In [5]:
# YOLO datasets
def prepare_yolo_dataset(splits, classes):
    import yaml
    
    for split in ['train', 'val', 'test']:
        (DATASET_YOLO / 'images' / split).mkdir(parents=True, exist_ok=True)
        (DATASET_YOLO / 'labels' / split).mkdir(parents=True, exist_ok=True)
    
    for split, items in splits.items():
        for item in tqdm(items, desc=f"YOLO {split}"):
            with open(item['image'], 'rb') as f:
                img = cv2.imdecode(np.frombuffer(f.read(), np.uint8), cv2.IMREAD_COLOR)
            h, w = img.shape[:2]
            
            img_save = DATASET_YOLO / 'images' / split / f"{item['json_stem']}.jpg"
            cv2.imwrite(str(img_save), img)
            
            bbox = item['bbox']
            x_center = (bbox[0] + bbox[2]) / 2 / w
            y_center = (bbox[1] + bbox[3]) / 2 / h
            width = (bbox[2] - bbox[0]) / w
            height = (bbox[3] - bbox[1]) / h
            
            label_save = DATASET_YOLO / 'labels' / split / f"{item['json_stem']}.txt"
            with open(label_save, 'w') as f:
                f.write(f"{item['label']} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
    
    # data.yaml
    with open(DATASET_YOLO / 'data.yaml', 'w', encoding='utf-8') as f:
        yaml.dump({
            'path': str(DATASET_YOLO),
            'train': 'images/train',
            'val': 'images/val',
            'test': 'images/test',
            'nc': len(classes),
            'names': classes
        }, f, allow_unicode=True)
    
    print(f" YOLO 데이터셋 준비 완료: {DATASET_YOLO}")

In [6]:
# EfficientDet 데이터셋
class EffDetDataset(Dataset):
    def __init__(self, data, img_size=512):
        self.data = data
        self.img_size = img_size
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        with open(item['image'], 'rb') as f:
            img = cv2.imdecode(np.frombuffer(f.read(), np.uint8), cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        img = cv2.resize(img, (self.img_size, self.img_size))
        img_tensor = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0
        
        bbox = item['bbox']
        bbox_scaled = [
            bbox[0] * self.img_size / w,
            bbox[1] * self.img_size / h,
            bbox[2] * self.img_size / w,
            bbox[3] * self.img_size / h
        ]
        
        return img_tensor, {
            'bbox': torch.tensor([bbox_scaled], dtype=torch.float32),
            'cls': torch.tensor([item['label']], dtype=torch.long),
            'img_scale': torch.tensor([1.0], dtype=torch.float32),
            'img_size': torch.tensor([self.img_size, self.img_size], dtype=torch.long)
        }

In [7]:
def collate_fn(batch):
    images = torch.stack([x[0] for x in batch])
    max_boxes = max([x[1]['bbox'].shape[0] for x in batch])
    
    bboxes, classes, scales, sizes = [], [], [], []
    for x in batch:
        bbox, cls = x[1]['bbox'], x[1]['cls']
        n = bbox.shape[0]
        if n < max_boxes:
            bbox = torch.cat([bbox, torch.zeros((max_boxes - n, 4))])
            cls = torch.cat([cls, torch.ones(max_boxes - n, dtype=torch.long) * -1])
        bboxes.append(bbox)
        classes.append(cls)
        scales.append(x[1]['img_scale'])
        sizes.append(x[1]['img_size'])
    
    return images, {
        'bbox': torch.stack(bboxes),
        'cls': torch.stack(classes),
        'img_scale': torch.stack(scales),
        'img_size': torch.stack(sizes)
    }

In [8]:
# Yolov5 모델
def train_yolo(data_yaml, epochs=15):
    print("\n YOLOv5 학습 시작")
    model = YOLO(str(YOLO_WEIGHTS_FILE))
    
    results = model.train(
        data=str(data_yaml),
        epochs=epochs,
        imgsz=640,
        batch=16,
        name='yolov5_freshness',
        device='0' if torch.cuda.is_available() else 'cpu',
        patience=10,
        workers=0,
        project=str(RESULT_DIR)
    )
    
    metrics = model.val(
        data=str(data_yaml),
        project=str(RESULT_DIR),
        name='yolov5_freshness'
    )
    yolo_metrics = {
        'mAP50': metrics.box.map50,
        'mAP50_95': metrics.box.map,
        'precision': metrics.box.mp,
        'recall': metrics.box.mr
    }
    
    print(f"  YOLOv5 학습 완료")
    print(f"  mAP@0.5: {yolo_metrics['mAP50']:.3f}")
    print(f"  mAP@0.5:0.95: {yolo_metrics['mAP50_95']:.3f}")
    
    return model, yolo_metrics

In [9]:
# COCO 형태변환
def create_coco_annotations(splits, classes):    
    coco_categories = []
    for idx, name in enumerate(classes): 
        coco_categories.append({
            "id": idx, 
            "name": name, 
            "supercategory": "freshness"
        })

    for split in ['train', 'val', 'test']:
        
        coco_images = []
        coco_annotations = []
        
        coco_data = {
            'info': {
                "description": f"Custom Dataset - {split} Set",
                "version": "1.0",
                "year": 2025,
                "contributor": "Contributor",
                "date_created": "2025/11/11"
            },
            'licenses': [{"id": 0, "name": "Unknown", "url": ""}],
            'categories': coco_categories,
            'images': coco_images,
            'annotations': coco_annotations
        }
        
        ann_id = 0
        
        for img_id, item in enumerate(splits[split]):
            try:
                with open(str(item['image']), 'rb') as f:
                    img = cv2.imdecode(np.frombuffer(f.read(), np.uint8), cv2.IMREAD_COLOR)
                h, w = img.shape[:2]
            except Exception as e:
                print(f"이미지 로드 실패 {item['image']}: {e}")
                continue
            
            coco_data['images'].append({
                'id': img_id,
                'file_name': str(item['image']),
                'width': w,
                'height': h
            })
            
            bbox = item['bbox']
            coco_data['annotations'].append({
                'id': ann_id,
                'image_id': img_id,
                'category_id': item['label'],
                'bbox': [bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]],
                'area': (bbox[2] - bbox[0]) * (bbox[3] - bbox[1]),
                'iscrowd': 0
            })
            ann_id += 1
        
        anno_path = RESULT_DIR / f'coco_{split}.json'
        with open(anno_path, 'w') as f:
            json.dump(coco_data, f, indent=4)
        
        print(f"COCO {split} annotations: {anno_path}")
    
    return RESULT_DIR / 'coco_test.json'

In [10]:
# efficientdet 모델
def train_efficientdet(splits, classes, epochs=50):
    print("\n EfficientDet 학습 시작")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_classes = len(classes)
    
    gt_anno_file = create_coco_annotations(splits, classes)

    train_loader = DataLoader(EffDetDataset(splits['train']), batch_size=4,
                             shuffle=True, collate_fn=collate_fn, num_workers=0)
    val_loader = DataLoader(EffDetDataset(splits['val']), batch_size=4,
                           collate_fn=collate_fn, num_workers=0)
    
    config = get_efficientdet_config('tf_efficientdet_d0')
    config.num_classes = num_classes
    config.image_size = (512, 512)
    
    model = DetBenchTrain(EfficientDet(config, pretrained_backbone=True), config)
    model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.0001)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
    
    best_loss = float('inf')
    patience_counter = 0
    max_patience = 10
    
    train_losses, val_losses = [], []
    val_recalls = []
    val_maps = []

    for epoch in range(epochs):
        # 학습
        model.train()
        train_loss = 0
        for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images = images.to(device)
            targets = {k: v.to(device) for k, v in targets.items()}
            
            optimizer.zero_grad()
            output = model(images, targets)
            loss = output['loss'] if isinstance(output, dict) else output
            
            # NaN 체크
            if torch.isnan(loss):
                print(" NaN loss detected")
                continue
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 10.0)
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        # 검증
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, targets in val_loader:
                images = images.to(device)
                targets = {k: v.to(device) for k, v in targets.items()}
                output = model(images, targets)
                loss = output['loss'] if isinstance(output, dict) else output
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        val_losses.append(val_loss) 
        print(f"Epoch {epoch+1}: Train={train_loss:.4f}, Val={val_loss:.4f}")

        # Early stopping
        if epoch == 0 or val_loss < best_loss:
            if val_loss < best_loss:
                best_loss = val_loss
            
            patience_counter = 0
            torch.save({
                'model_state_dict': model.model.state_dict(),
                'config': config
            }, RESULT_DIR / 'efficientdet_best.pth')
            print(f" Saved (Loss: {best_loss:.4f})")
        else:
            patience_counter += 1
            if patience_counter >= max_patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
        
        scheduler.step()
        effdet_metrics = evaluate_efficientdet_coco(model, config, splits, classes, device, gt_anno_file)
        val_maps.append(effdet_metrics['mAP50_95'])
        val_recalls.append(effdet_metrics['recall'])    
    print(f" EfficientDet 학습 완료 (Best Loss: {best_loss:.4f})")

    #loss
    plt.figure(figsize=(8, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title("EfficientDet Training Loss Curve")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(RESULT_DIR / "efficientdet_loss_curve.png")
    plt.close()
    print(f"EfficientDet 학습 곡선 저장됨: {RESULT_DIR / 'efficientdet_loss_curve.png'}")
          
    # mAP/recall
    plt.figure(figsize=(8, 5))
    plt.plot(val_maps, label='Validation mAP50_95')
    plt.plot(val_recalls, label='Validation Recall')
    plt.title("EfficientDet Validation mAP/Recall Curve")
    plt.xlabel("Epoch")
    plt.ylabel("Score")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(RESULT_DIR / "efficientdet_map_curve.png")
    plt.close()
    print(f"EfficientDet mAP/Recall 곡선 저장됨: {RESULT_DIR / 'efficientdet_map_curve.png'}")

    return model, effdet_metrics

In [ ]:
# COCO 평가지표
def evaluate_efficientdet_coco(model, config, splits, classes, device, gt_anno_file):
    print("\n EfficientDet 평가 중 (COCO API)")
    
    if not COCO_AVAILABLE:
        print(" COCO API 없음.")
        return evaluate_efficientdet_simple(None, config, splits, device)

    checkpoint = torch.load(RESULT_DIR / 'efficientdet_best.pth', weights_only=False)
    net = EfficientDet(config, pretrained_backbone=False)
    net.load_state_dict(checkpoint['model_state_dict'])
    
    from effdet import DetBenchPredict
    bench = DetBenchPredict(net)
    bench.eval()
    bench.to(device)
    
    coco_gt = COCO(str(gt_anno_file))
    
    results = []
    test_data = splits['test']
    
    vis_dir = RESULT_DIR / 'efficientdet_predictions'
    vis_dir.mkdir(exist_ok=True)
    
    # 디버깅
    total_detections = 0
    detection_scores = []
    debug_output = True
    
    for img_id, item in enumerate(tqdm(test_data, desc="Predicting")):
        with open(item['image'], 'rb') as f:
            img = cv2.imdecode(np.frombuffer(f.read(), np.uint8), cv2.IMREAD_COLOR)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img_rgb.shape[:2]
        
        # 추론
        img_resized = cv2.resize(img_rgb, (512, 512))
        img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
        img_tensor = img_tensor.unsqueeze(0).to(device)
        
        with torch.no_grad():
            detections = bench(img_tensor)
            
            # 디버깅
            if debug_output:
                print(f"\n[DEBUG] DetBenchPredict 출력 타입: {type(detections)}")
                if hasattr(detections, 'shape'):
                    print(f"[DEBUG] 출력 shape: {detections.shape}")
                    if detections.shape[0] > 0:
                        print(f"[DEBUG] 샘플 (첫 5개):\n{detections[0][:5]}")
                else:
                    print(f"[DEBUG] 예상치 못한 출력")
                debug_output = False
        
        vis_img = img.copy()
        
        conf_thresh = 0.001
        
        if detections is not None and hasattr(detections, 'shape') and len(detections.shape) == 3:
            det = detections[0].cpu()  
            
            for i in range(det.shape[0]):
                if det.shape[1] >= 6:
                    score = float(det[i, 4])
                    class_id = int(det[i, 5])
                elif det.shape[1] == 5:
                    score = float(det[i, 4])
                    class_id = 0  
                else:
                    continue
                
                if score < conf_thresh:
                    continue
                
                detection_scores.append(score)
                total_detections += 1
                
                x1 = float(det[i, 0]) * w / 512
                y1 = float(det[i, 1]) * h / 512
                x2 = float(det[i, 2]) * w / 512
                y2 = float(det[i, 3]) * h / 512
                
                if class_id < 0 or class_id >= len(classes):
                    continue
                if x2 <= x1 or y2 <= y1:
                    continue
                
                results.append({
                    'image_id': img_id,
                    'category_id': class_id,
                    'bbox': [x1, y1, x2 - x1, y2 - y1],
                    'score': score
                })
            
                if score > 0.01:

                    # 좌표 클리핑 추가
                    x1_clipped = max(0, min(int(x1), w))
                    y1_clipped = max(0, min(int(y1), h))
                    x2_clipped = max(0, min(int(x2), w))
                    y2_clipped = max(0, min(int(y2), h))
                    
                    # 유효한 bbox인지 확인
                    if x2_clipped > x1_clipped and y2_clipped > y1_clipped:
                        cv2.rectangle(vis_img, (x1_clipped, y1_clipped), (x2_clipped, y2_clipped), (0, 255, 0), 2)
                        label = f"{classes[class_id]}: {score:.2f}"
                        org = (x1_clipped, max(10, y1_clipped - 10))
                        cv2.putText(vis_img, str(label), org, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
            
            # 예측 이미지 저장
            cv2.imwrite(str(vis_dir / f"{item['json_stem']}_pred.jpg"), vis_img)
    
    print(f"\n[DEBUG] 총 detections: {total_detections}개")
    if detection_scores:
        print(f"[DEBUG] Score 범위: {min(detection_scores):.4f} ~ {max(detection_scores):.4f}")
        print(f"[DEBUG] Score 평균: {np.mean(detection_scores):.4f}")
    print(f"[DEBUG] 최종 예측 결과: {len(results)}개")
    
    if not results:
        print("예측 결과 없음!")
        print("모델이 아무것도 검출하지 못했습니다.")
        print("해결 방법:")
        print("1) epochs를 100 이상으로 증가")
        print("2) learning rate 0.01로 증가")
        print("3) 데이터셋이 너무 작은지 확인 (최소 100장 이상 권장)")
        print("4) 클래스 개수 확인")
        return evaluate_efficientdet_simple(checkpoint, config, splits, device)
    
    # 예측 결과 저장
    pred_file = RESULT_DIR / 'coco_predictions.json'
    with open(pred_file, 'w') as f:
        json.dump(results, f)
    
    # COCO 평가
    try:
        coco_dt = coco_gt.loadRes(str(pred_file))
        coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()
        
        metrics = {
            'mAP50_95': coco_eval.stats[0],
            'mAP50': coco_eval.stats[1],
            'mAP75': coco_eval.stats[2],
            'precision': coco_eval.stats[0],
            'recall': coco_eval.stats[8]
        }
    except Exception as e:
        print(f"COCO 평가 오류: {e}")
        return evaluate_efficientdet_simple(checkpoint, config, splits, device)
    
    print(f" EfficientDet 평가 완료 (COCO API)")
    print(f"  mAP@0.5: {metrics['mAP50']:.3f}")
    print(f"  mAP@0.5:0.95: {metrics['mAP50_95']:.3f}")
    
    return metrics

In [12]:
def evaluate_efficientdet_simple(model, config, splits, device):
    print("\n EfficientDet 평가 중")
    
    checkpoint = torch.load(RESULT_DIR / 'efficientdet_best.pth', weights_only=False)
    net = EfficientDet(config, pretrained_backbone=False)
    net.load_state_dict(checkpoint['model_state_dict'])
    predictor = DetBenchPredict(net).to(device).eval()
    
    test_data = splits['test']
    
    def calculate_iou(box1, box2):
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        inter = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - inter
        
        return inter / union if union > 0 else 0
    
    correct_50 = 0
    all_ious = []
    
    for item in tqdm(test_data, desc="Evaluating"):
        with open(item['image'], 'rb') as f:
            img = cv2.imdecode(np.frombuffer(f.read(), np.uint8), cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        img_resized = cv2.resize(img, (512, 512))
        img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
        img_tensor = img_tensor.unsqueeze(0).to(device)
        
        with torch.no_grad():
            pred = predictor(img_tensor)
        
        gt_box = item['bbox']
        gt_label = item['label']
        
        best_iou = 0
        
        if len(pred) > 0 and pred[0].shape[0] > 0:
            pred_np = pred[0].cpu().numpy()
            for box in pred_np:
                if box[4] < 0.1:
                    continue
                
                pred_box = [
                    box[0] * w / 512, box[1] * h / 512,
                    box[2] * w / 512, box[3] * h / 512
                ]
                pred_label = int(box[5]) if len(box) > 5 else int(box[4])
                
                if pred_label == gt_label:
                    iou = calculate_iou(gt_box, pred_box)
                    best_iou = max(best_iou, iou)
        
        all_ious.append(best_iou)
        if best_iou >= 0.5:
            correct_50 += 1
    
    total = len(test_data)
    mAP_50 = correct_50 / total if total > 0 else 0
    
    mAP_50_95_sum = 0
    for thresh in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:
        correct = sum([1 for iou in all_ious if iou >= thresh])
        mAP_50_95_sum += (correct / total if total > 0 else 0)
    mAP_50_95 = mAP_50_95_sum / 10
    
    metrics = {
        'mAP50': mAP_50,
        'mAP50_95': mAP_50_95,
        'precision': mAP_50,
        'recall': mAP_50
    }
    
    print(f"  EfficientDet 평가 완료")
    print(f"  mAP@0.5: {metrics['mAP50']:.3f}")
    print(f"  mAP@0.5:0.95: {metrics['mAP50_95']:.3f}")
    
    return metrics

In [13]:
# 시각화
def visualize_comparison(yolo_metrics, effdet_metrics):
    metrics_names = ['mAP@0.5', 'mAP@0.5:0.95', 'Precision', 'Recall']
    yolo_values = [
        yolo_metrics['mAP50'],
        yolo_metrics['mAP50_95'],
        yolo_metrics['precision'],
        yolo_metrics['recall']
    ]
    effdet_values = [
        effdet_metrics['mAP50'],
        effdet_metrics['mAP50_95'],
        effdet_metrics['precision'],
        effdet_metrics['recall']
    ]
    
    x = np.arange(len(metrics_names))
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(12, 6))
    bars1 = ax.bar(x - width/2, yolo_values, width, label='YOLOv5', color='red')
    bars2 = ax.bar(x + width/2, effdet_values, width, label='EfficientDet', color='blue')
    
    ax.set_xlabel('Metrics', fontsize=12, fontweight='bold')
    ax.set_ylabel('Score', fontsize=12, fontweight='bold')
    ax.set_title('YOLOv5 vs EfficientDet Performance Comparison', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics_names)
    ax.legend(fontsize=11)
    ax.set_ylim(0, 1.1)
    ax.grid(axis='y', alpha=0.3)
    
    # 값 표시
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.3f}',
                   ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig(RESULT_DIR / 'performance_comparison.png', dpi=300, bbox_inches='tight')
    print(f"\n 비교 그래프 저장: {RESULT_DIR / 'performance_comparison.png'}")
    plt.close()


In [14]:
def main():
    print("\n" + "="*60)
    print("YOLOv5 vs EfficientDet 성능 비교")
    print("="*60)
    
    # 1. 데이터 전처리
    print("\n1. 데이터 전처리 중...")
    splits, classes = preprocess_data()
    if not classes:
        print("데이터셋 없음")
        return
    print(f"클래스 ({len(classes)}개): {classes}")
    
    # 2. YOLOv5
    prepare_yolo_dataset(splits, classes)
    yolo_model, yolo_metrics = train_yolo(DATASET_YOLO / 'data.yaml', epochs=10)

    # 3. EfficientDet
    effdet_model, effdet_metrics = train_efficientdet(splits, classes, epochs=50)
    
    # 4. 결과 비교
    print("\n" + "="*60)
    print("최종 성능 비교")
    print("="*60)
    print(f"\n{'Metric':<20} {'YOLOv5':<15} {'EfficientDet':<15} {'Difference':<15}")
    print("-"*65)
    print(f"{'mAP@0.5':<20} {yolo_metrics['mAP50']:<15.3f} {effdet_metrics['mAP50']:<15.3f} {yolo_metrics['mAP50']-effdet_metrics['mAP50']:+.3f}")
    print(f"{'mAP@0.5:0.95':<20} {yolo_metrics['mAP50_95']:<15.3f} {effdet_metrics['mAP50_95']:<15.3f} {yolo_metrics['mAP50_95']-effdet_metrics['mAP50_95']:+.3f}")
    print(f"{'Precision':<20} {yolo_metrics['precision']:<15.3f} {effdet_metrics['precision']:<15.3f} {yolo_metrics['precision']-effdet_metrics['precision']:+.3f}")
    print(f"{'Recall':<20} {yolo_metrics['recall']:<15.3f} {effdet_metrics['recall']:<15.3f} {yolo_metrics['recall']-effdet_metrics['recall']:+.3f}")
    
    # 승자 결정
    if yolo_metrics['mAP50'] > effdet_metrics['mAP50']:
        winner = "YOLOv5"
        diff = yolo_metrics['mAP50'] - effdet_metrics['mAP50']
    elif effdet_metrics['mAP50'] > yolo_metrics['mAP50']:
        winner = "EfficientDet"
        diff = effdet_metrics['mAP50'] - yolo_metrics['mAP50']
    else:
        winner = "동점"
        diff = 0
    
    print(f"\n{'='*65}")
    if winner != "동점":
        print(f"🏆 {winner}가 {diff:.3f}만큼 더 높은 mAP@0.5를 달성했습니다!")
    else:
        print(f"🏆 두 모델이 동일한 성능을 보였습니다!")
    print(f"{'='*65}")
    
    # 5. 시각화
    visualize_comparison(yolo_metrics, effdet_metrics)
    
    # 6. 결과 저장
    results_summary = {
        'yolo': yolo_metrics,
        'efficientdet': effdet_metrics,
        'winner': winner,
        'classes': classes
    }
    
    with open(RESULT_DIR / 'comparison_results.json', 'w', encoding='utf-8') as f:
        json.dump(results_summary, f, indent=2, ensure_ascii=False)
    
    print(f"\n 결과 저장 위치:")
    print(f"  - 전체 결과: {RESULT_DIR}")
    print(f"  - YOLOv5 학습: {RESULT_DIR / 'yolov5_freshness'}")
    print(f"  - EfficientDet 예측: {RESULT_DIR / 'efficientdet_predictions'}")
    print(f"  - 비교 그래프: {RESULT_DIR / 'performance_comparison.png'}")
    print(f"  - 결과 요약: {RESULT_DIR / 'comparison_results.json'}")

    # 성능 비교 그래프
    labels = ['mAP@0.5', 'mAP@0.5:0.95']
    yolo_scores = [yolo_metrics['mAP50'], yolo_metrics['mAP50_95']]
    effdet_scores = [effdet_metrics['mAP50'], effdet_metrics['mAP50_95']]

    x = np.arange(len(labels))
    width = 0.35

    plt.figure(figsize=(6, 4))
    plt.bar(x - width/2, yolo_scores, width, label='YOLOv5')
    plt.bar(x + width/2, effdet_scores, width, label='EfficientDet')

    plt.ylabel('Score')
    plt.title('YOLOv5 vs EfficientDet Performance')
    plt.xticks(x, labels)
    plt.legend()
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(RESULT_DIR / 'comparison_graph.png')
    plt.close()
    print(f" 성능 비교 그래프 저장됨: {RESULT_DIR / 'comparison_graph.png'}")



if __name__ == "__main__":
    main()


YOLOv5 vs EfficientDet 성능 비교

1. 데이터 전처리 중...


Loading test: 100%|██████████| 7/7 [00:00<00:00, 5327.55it/s]


Dataset: train=56, val=7, test=7
클래스 (7개): ['사과_상', '감_특', '배_보통', '감_상', '사과_보통', '감_보통', '사과_특']


YOLO test: 100%|██████████| 7/7 [00:00<00:00, 58.90it/s]


 YOLO 데이터셋 준비 완료: /Users/handaeseong/dev/data-engineer/mini-project-2-fruits/model/yolov5

 YOLOv5 학습 시작
New https://pypi.org/project/ultralytics/8.3.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.225 🚀 Python-3.10.19 torch-2.9.0 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/handaeseong/dev/data-engineer/mini-project-2-fruits/model/yolov5/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_

...


[DEBUG] DetBenchPredict 출력 타입: <class 'torch.Tensor'>
[DEBUG] 출력 shape: torch.Size([1, 100, 6])
[DEBUG] 샘플 (첫 5개):
tensor([[ 2.3300e+00,  2.9410e+00,  5.1574e+02,  5.1491e+02,  7.4509e-01,  1.0000e+00],
        [-2.2988e+00, -4.5622e+00,  5.1427e+02,  5.0955e+02,  1.8534e-01,  3.0000e+00],
        [ 4.9320e+01, -1.9204e+02,  5.9105e+02,  3.1632e+02,  7.0625e-02,  4.0000e+00],
        [-2.7047e+02,  1.4505e+02,  3.3846e+02,  5.7957e+02,  5.9871e-02,  1.0000e+00],
        [-2.8902e+02, -3.9463e+02,  4.3499e+02,  6.8444e+02,  5.9848e-02,  3.0000e+00]])


Predicting: 100%|██████████| 7/7 [00:02<00:00,  2.72it/s]



[DEBUG] 총 detections: 700개
[DEBUG] Score 범위: 0.0097 ~ 0.9965
[DEBUG] Score 평균: 0.0298
[DEBUG] 최종 예측 결과: 694개
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.475
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.500
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.500
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.475
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.475
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.475
 Average Recall     (AR) @[ IoU=0.50

Epoch 49/50: 100%|██████████| 14/14 [16:26<00:00, 70.44s/it] 


Epoch 49: Train=0.0169, Val=0.1065
Early stopping at epoch 49
 EfficientDet 학습 완료 (Best Loss: 0.0675)
EfficientDet 학습 곡선 저장됨: /Users/handaeseong/dev/data-engineer/mini-project-2-fruits/data/processed/results_comparison/efficientdet_loss_curve.png
EfficientDet mAP/Recall 곡선 저장됨: /Users/handaeseong/dev/data-engineer/mini-project-2-fruits/data/processed/results_comparison/efficientdet_map_curve.png

최종 성능 비교

Metric               YOLOv5          EfficientDet    Difference     
-----------------------------------------------------------------
mAP@0.5              0.967           0.500           +0.467
mAP@0.5:0.95         0.946           0.475           +0.471
Precision            0.725           0.475           +0.250
Recall               0.750           0.475           +0.275

🏆 YOLOv5가 0.467만큼 더 높은 mAP@0.5를 달성했습니다!

 비교 그래프 저장: /Users/handaeseong/dev/data-engineer/mini-project-2-fruits/data/processed/results_comparison/performance_comparison.png

 결과 저장 위치:
  - 전체 결과: /Users/handaeseong